<a href="https://colab.research.google.com/github/alex-nuclearboy/goit-python-ds-project/blob/main/toxic_bert_model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## BERT model trained on the `train_data_cleaned` dataset

In [1]:
!pip install torch
!pip install transformers

In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
# Download the dataset from Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Path to a clean dataset
dataset_path = '/content/drive/My Drive/toxic_comments_data/train_data_cleaned.csv'

# Load the dataset into a DataFrame
df_train = pd.read_csv(dataset_path)

Mounted at /content/drive


In [4]:
print(df_train.columns)

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate', 'token_count'],
      dtype='object')


In [5]:
# Split data into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(
    df_train['comment_text'],
    df_train.iloc[:, 2:8],
    test_size=0.20,
    random_state=42
)

In [6]:
print(f'Training data shape: {train_data.shape}')
print(f'Training labels shape: {train_labels.shape}')
print(f'Validation data shape: {val_data.shape}')
print(f'Validation labels shape: {val_labels.shape}')

Training data shape: (127656,)
Training labels shape: (127656, 6)
Validation data shape: (31915,)
Validation labels shape: (31915, 6)


In [7]:
# Token and Encode Function
def tokenize_and_encode(tokenizer, comments, labels, max_length=512):
	# Initialize empty lists to store tokenized inputs and attention masks
	input_ids = []
	attention_masks = []

	# Iterate through each comment in the 'comments' list
	for comment in comments:

		# Tokenize and encode the comment using the ALBERT tokenizer
		encoded_dict = tokenizer.encode_plus(
			comment,

			# Add special tokens like [CLS] and [SEP]
			add_special_tokens=True,

			# Truncate or pad the comment to 'max_length'
			max_length=max_length,
      padding='max_length',
      truncation=True,

			# Return attention mask to mask padded tokens
			return_attention_mask=True,

			# Return PyTorch tensors
			return_tensors='pt'
		)

		# Append the tokenized input and attention mask to their respective lists
		input_ids.append(encoded_dict['input_ids'])
		attention_masks.append(encoded_dict['attention_mask'])

	# Concatenate the tokenized inputs and attention masks into tensors
	input_ids = torch.cat(input_ids, dim=0)
	attention_masks = torch.cat(attention_masks, dim=0)

	# Convert the labels to a PyTorch tensor with the data type float32
	labels = torch.tensor(labels, dtype=torch.float32)

	# Return the tokenized inputs, attention masks, and labels as PyTorch tensors
	return input_ids, attention_masks, labels

In [8]:
# Token Initialization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
# Model Initialization
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Check for NaN values in train_data
print("NaN values in train_data:")
print(train_data.isnull().sum())

# Drop rows with NaN values in train_data
train_data_cleaned = train_data.dropna()

# Filter train_labels to match the cleaned train_data
train_labels_cleaned = train_labels.loc[train_data_cleaned.index]

# Recheck for NaN values after cleaning
print("NaN values in train_data_cleaned:")
print(train_data_cleaned.isnull().sum())
print("NaN values in train_labels_cleaned:")
print(train_labels_cleaned.isnull().sum())

NaN values in train_data:
7
NaN values in train_data_cleaned:
0
NaN values in train_labels_cleaned:
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64


In [11]:
# Check for NaN values in val_data
print("NaN values in val_data:")
print(val_data.isnull().sum())

# Drop rows with NaN values in val_data
val_data_cleaned = val_data.dropna()

# Filter val_labels to match the cleaned val_data
val_labels_cleaned = val_labels.loc[val_data_cleaned.index]

# Recheck for NaN values after cleaning
print("NaN values in val_data_cleaned:")
print(val_data_cleaned.isnull().sum())
print("NaN values in val_labels_cleaned:")
print(val_labels_cleaned.isnull().sum())

NaN values in val_data:
2
NaN values in val_data_cleaned:
0
NaN values in val_labels_cleaned:
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64


In [12]:
# Set the device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

In [13]:
# Tokenize and Encode the comments and labels for the training set
input_ids, attention_masks, labels = tokenize_and_encode(
    tokenizer,
    train_data_cleaned,
    train_labels_cleaned.values
)

In [14]:
# Tokenize and Encode the comments and labels for the validation set
val_input_ids, val_attention_masks, val_labels = tokenize_and_encode(
    tokenizer,
    val_data_cleaned,
    val_labels_cleaned.values
)

In [15]:
print('Training Comments :',train_data_cleaned.shape)
print('Input Ids         :',input_ids.shape)
print('Attention Mask    :',attention_masks.shape)
print('Labels            :',labels.shape)

Training Comments : (127649,)
Input Ids         : torch.Size([127649, 512])
Attention Mask    : torch.Size([127649, 512])
Labels            : torch.Size([127649, 6])


In [16]:
# Check an encoded text with the corresponding text and labels (e.g., comment #25)
i = 25
print('Example comment:',train_data_cleaned.values[i])
print('\nInput Ids:\n',input_ids[i])
print('\nDecoded Ids:\n',tokenizer.decode(input_ids[i]))
print('\nAttention Mask:\n',attention_masks[i])
print('\nLabels:',labels[i])

Example comment: you, and no one else, have explained why this one word is vandalism. it cetainly appiles to him, and you are now saying that people have to decide wheter it applies by some sort of vote that is certainly not neutrality. on what basis is this word vanalism

Input Ids:
 tensor([  101,  2017,  1010,  1998,  2053,  2028,  2842,  1010,  2031,  4541,
         2339,  2023,  2028,  2773,  2003,  3158,  9305,  2964,  1012,  2009,
         8292, 18249,  2135, 10439,  9463,  2015,  2000,  2032,  1010,  1998,
         2017,  2024,  2085,  3038,  2008,  2111,  2031,  2000,  5630,  1059,
        27065,  2121,  2009, 12033,  2011,  2070,  4066,  1997,  3789,  2008,
         2003,  5121,  2025, 21083,  1012,  2006,  2054,  3978,  2003,  2023,
         2773,  3158, 13911,  2213,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            

In [17]:
from torch.utils.data import DataLoader, TensorDataset

# Creating DataLoader
batch_size = 32

# Training set
train_dataset = TensorDataset(input_ids, attention_masks, labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Validation set
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [18]:
# Check the train_loader data
print('Batch Size :',train_loader.batch_size)
Batch = next(iter(train_loader))
print('Each Input ids shape :', Batch[0].shape)
print('Input ids :\n', Batch[0][0])
print('Corresponding Decoded text:\n', tokenizer.decode(Batch[0][0]))
print('Corresponding Attention Mask :\n', Batch[1][0])
print('Corresponding Label:', Batch[2][0])

Batch Size : 32
Each Input ids shape : torch.Size([32, 512])
Input ids :
 tensor([  101,  5310,  2937,  1012, 11161,  1010,  2009,  3544,  2008,  2017,
         2123,  2102,  2941,  2113,  1996,  3574,  1997,  1996,  2773, 10465,
         2017,  2224,  2009,  8989,  8231,  2007,  3484,  1012,  4312,  1010,
         1996,  3277,  2003,  4895,  6072, 16116,  2119,  2182,  1998,  2006,
         1996, 15631,  5060,  6277,  1010,  2061,  2049,  6135, 15884,  2005,
         2017,  2000,  2022,  8223,  6562,  3431,  2083,  1999,  1996, 12507,
         1012,  2004,  1045,  2056,  2077,  1010,  7036,  2135,  1998, 14044,
         2135,  4964,  1037, 18700,  3277,  2009,  2001,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     

In [19]:
from transformers import AdamW

# Optimizer setup
optimizer = AdamW(model.parameters(), lr=2e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
# Function to train the model
def train_model(model, train_loader, optimizer, device, num_epochs):
    # Loop through the specified number of epochs
    for epoch in range(num_epochs):
        # Set the model to training mode
        model.train()
        # Initialize total loss for the current epoch
        total_loss = 0

        # Loop through the batches in the training data
        for batch in train_loader:
            input_ids, attention_mask, labels = [t.to(device) for t in batch]

            optimizer.zero_grad() # Clear the gradients from the previous step to prevent accumulation

            outputs = model(
                input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        model.eval()  # Set the model to evaluation mode
        val_loss = 0

        # Disable gradient computation during validation
        with torch.no_grad():
            for batch in val_loader:
                input_ids, attention_mask, labels = [
                    t.to(device) for t in batch]

                outputs = model(
                    input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                val_loss += loss.item()
        # Print the average loss for the current epoch
        print(
            f'Epoch {epoch+1}, Training Loss: {total_loss/len(train_loader)},Validation loss:{val_loss/len(val_loader)}')

In [21]:
# Call the function to train the model
train_model(model, train_loader, optimizer, device, num_epochs=3)

Epoch 1, Training Loss: 0.050322253651214406,Validation loss:0.03806221094079276
Epoch 2, Training Loss: 0.03311697171764964,Validation loss:0.03869582717788671
Epoch 3, Training Loss: 0.026044462307036522,Validation loss:0.04045161419757012


## Test the model on the `test_cleaned` dataset

In [22]:
# Path to a clean test dataset
test_dataset_path = '/content/drive/My Drive/toxic_comments_data/test_cleaned.csv'

# Load the dataset into a DataFrame
df_test = pd.read_csv(test_dataset_path)

In [23]:
df_test.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0001ea8717f6de06,thank you for understanding. i think very high...,0,0,0,0,0,0
1,000247e83dcc1211,dear god this site is horrible.,0,0,0,0,0,0
2,0002f87b16116a7f,somebody will invariably try to add religion r...,0,0,0,0,0,0
3,0003e1cccfd5a40a,it says it right there that it is a type. the ...,0,0,0,0,0,0
4,00059ace3e3e9a53,"before adding a new product to the list, make ...",0,0,0,0,0,0


In [24]:
# Extract the comment texts and labels from the test DataFrame
test_data = df_test['comment_text']
test_labels = df_test.iloc[:, 2:]

In [25]:
print(f'Test data shape: {test_data.shape}')
print(f'Test labels shape: {test_labels.shape}')

Test data shape: (63978,)
Test labels shape: (63978, 6)


In [26]:
# Check for NaN values in test_data
print("NaN values in test_data:")
print(test_data.isnull().sum())

# Drop rows with NaN values in test_data
test_data_cleaned = test_data.dropna()

# Filter test_labels to match the cleaned test_data
test_labels_cleaned = test_labels.loc[test_data_cleaned.index]

# Recheck for NaN values after cleaning
print("NaN values in test_data_cleaned:")
print(test_data_cleaned.isnull().sum())
print("NaN values in test_labels_cleaned:")
print(test_labels_cleaned.isnull().sum())

NaN values in test_data:
501
NaN values in test_data_cleaned:
0
NaN values in test_labels_cleaned:
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64


In [27]:
# Tokenize and Encode the comments and labels for the test set
test_input_ids, test_attention_masks, test_labels = tokenize_and_encode(
    tokenizer,
    test_data_cleaned,
    test_labels_cleaned.values
)

In [28]:
# Creating DataLoader for the testing dataset
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [29]:
# Evaluate the model
def evaluate_model(model, test_loader, device, label_names):
    model.eval()  # Set the model to evaluation mode

    true_labels = []  # List to store true labels
    predicted_probs = []  # List to store predicted probabilities

    # Disable gradient computation for evaluation
    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask, labels = [t.to(device) for t in batch]

            # Get model's predictions
            outputs = model(input_ids, attention_mask=attention_mask)
            # Apply sigmoid for multilabel classification to get probabilities
            predicted_probs_batch = torch.sigmoid(outputs.logits)
            predicted_probs.append(predicted_probs_batch.cpu().numpy())

            # Store true labels for later evaluation
            true_labels_batch = labels.cpu().numpy()
            true_labels.append(true_labels_batch)

    # Combine predictions and labels for evaluation
    true_labels = np.concatenate(true_labels, axis=0)
    predicted_probs = np.concatenate(predicted_probs, axis=0)

    # Apply threshold for binary classification (0.5 is commonly used)
    predicted_labels = (predicted_probs > 0.5).astype(int)

    # Calculate evaluation metrics for overall performance
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels, average='micro')
    recall = recall_score(true_labels, predicted_labels, average='micro')
    f1 = f1_score(true_labels, predicted_labels, average='micro')

    # Calculate evaluation metrics for each label (class-specific)
    precision_per_label = precision_score(true_labels, predicted_labels, average=None)
    recall_per_label = recall_score(true_labels, predicted_labels, average=None)
    f1_per_label = f1_score(true_labels, predicted_labels, average=None)

    # Print the overall evaluation metrics
    print(f'Overall Accuracy: {accuracy:.4f}')
    print(f'Overall Precision: {precision:.4f}')
    print(f'Overall Recall: {recall:.4f}')
    print(f'Overall F1 Score: {f1:.4f}')

    # Print evaluation metrics for each label
    print("\nEvaluation per label:")
    for i, label in enumerate(label_names):
        print(f'{label}:')
        print(f'  Precision: {precision_per_label[i]:.4f}')
        print(f'  Recall: {recall_per_label[i]:.4f}')
        print(f'  F1 Score: {f1_per_label[i]:.4f}')

In [30]:
# List of labels in the dataset
label_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Call the function to evaluate the model on the test data
evaluate_model(model, test_loader, device, label_names)

Overall Accuracy: 0.8624
Overall Precision: 0.5814
Overall Recall: 0.7711
Overall F1 Score: 0.6630

Evaluation per label:
toxic:
  Precision: 0.5054
  Recall: 0.9144
  F1 Score: 0.6510
severe_toxic:
  Precision: 0.3919
  Recall: 0.3196
  F1 Score: 0.3520
obscene:
  Precision: 0.6695
  Recall: 0.7628
  F1 Score: 0.7131
threat:
  Precision: 0.4506
  Recall: 0.5024
  F1 Score: 0.4751
insult:
  Precision: 0.7413
  Recall: 0.6510
  F1 Score: 0.6932
identity_hate:
  Precision: 0.7450
  Recall: 0.4759
  F1 Score: 0.5808


## Save the model

In [31]:
import os

# Define the output directory for the model and tokenizer
output_dir = "/content/drive/My Drive/toxic_bert_model_new"

# Create the directory if it does not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the model's state dictionary and configuration
model.save_pretrained(output_dir)

# Save the tokenizer's configuration and vocabulary
tokenizer.save_pretrained(output_dir)

# Save the model weights separately
torch.save(model.state_dict(), os.path.join(output_dir, 'toxic_bert_model_new_weights.pth'))

In [32]:
from safetensors.torch import save_file

# Save the model in safetensors format
save_file(model.state_dict(), os.path.join(output_dir, 'model.safetensors'))

In [33]:
import pickle

with open(os.path.join(output_dir, 'toxic_bert_model_new.pkl'), 'wb') as fh:
    pickle.dump(model, fh)